In [632]:
import yfinance as yf
import plotly.graph_objects as go
from ta import add_all_ta_features
from ta.utils import dropna
import numpy as np
# berkshire hathaway inc. (b stock)
msft = yf.Ticker('BNTX')

hist = msft.history(period="7d", interval="1m", auto_adjust=True)

hist = hist.drop(columns=['Dividends', 'Stock Splits'])

hist = dropna(hist)
#hist = hist.dropna()
hist = hist.reset_index()

hist

,Datetime,Open,High,Low,Close,Volume
0,2021-04-08 09:30:00-04:00,113.269997,113.955002,113.099998,113.848099,26082.0
1,2021-04-08 09:31:00-04:00,113.760002,114.000000,113.760002,113.940002,5149.0
2,2021-04-08 09:32:00-04:00,114.025002,114.025002,114.025002,114.025002,123.0
3,2021-04-08 09:33:00-04:00,114.169998,114.169998,113.879997,114.080002,1296.0
4,2021-04-08 09:34:00-04:00,114.160004,114.399902,113.880096,113.880096,4579.0
...,...,...,...,...,...,...
2525,2021-04-16 13:32:00-04:00,148.039902,148.194000,147.550003,148.194000,10328.0
2526,2021-04-16 13:33:00-04:00,148.164993,148.164993,147.550003,147.550003,4608.0
2527,2021-04-16 13:34:00-04:00,147.809998,147.809998,147.470001,147.710007,11533.0
2528,2021-04-16 13:35:00-04:00,147.544998,147.619995,147.429993,147.619995,4330.0


In [633]:
hist = add_all_ta_features(hist, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
hist

/Users/leon/opt/anaconda3/lib/python3.7/site-packages/ta/trend.py:768: RuntimeWarning:

invalid value encountered in double_scalars

/Users/leon/opt/anaconda3/lib/python3.7/site-packages/ta/trend.py:772: RuntimeWarning:

invalid value encountered in double_scalars



,Datetime,Open,High,Low,Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
0,2021-04-08 09:30:00-04:00,113.269997,113.955002,113.099998,113.848099,26082.0,1.955981e+04,26082.0,0.749935,0.000000,...,-12.503235,0.000000,113.848099,0.000000,0.000000,0.000000,0.000000,-10.786397,0.000000,0.000000
1,2021-04-08 09:31:00-04:00,113.760002,114.000000,113.760002,113.940002,5149.0,2.213439e+04,31231.0,0.708732,473.212082,...,-6.666384,0.000000,113.891434,0.000000,-6.807065,-1.361413,-5.445652,0.080725,0.080692,0.080725
2,2021-04-08 09:32:00-04:00,114.025002,114.025002,114.025002,114.025002,123.0,2.213439e+04,31354.0,0.705951,407.103911,...,-0.000000,0.000000,113.952827,0.000000,-14.786866,-4.046504,-10.740363,0.074600,0.074572,0.155385
3,2021-04-08 09:33:00-04:00,114.169998,114.169998,113.879997,114.080002,1296.0,2.262602e+04,32650.0,0.692987,359.129123,...,-8.410875,0.000000,114.011667,0.000000,-21.538663,-7.544936,-13.993728,0.048235,0.048224,0.203695
4,2021-04-08 09:34:00-04:00,114.160004,114.399902,113.880096,113.880096,4579.0,1.804702e+04,28071.0,0.484757,177.058276,...,-39.988027,0.000000,113.952384,0.000000,-25.981648,-11.232278,-14.749370,-0.175233,-0.175386,0.028106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525,2021-04-16 13:32:00-04:00,148.039902,148.194000,147.550003,148.194000,10328.0,1.415370e+06,3268649.0,-0.162144,-1293.105873,...,-44.157080,-1.814184,148.246959,-0.186297,-2.783250,7.084800,-9.868050,0.293723,0.293293,30.168182
2526,2021-04-16 13:33:00-04:00,148.164993,148.164993,147.550003,147.550003,4608.0,1.410762e+06,3264041.0,-0.170300,-1532.310614,...,-78.813293,-1.841165,148.237921,-0.148881,-7.219562,4.223927,-11.443490,-0.434564,-0.435511,29.602518
2527,2021-04-16 13:34:00-04:00,147.809998,147.809998,147.470001,147.710007,11533.0,1.415511e+06,3275574.0,-0.139597,-1049.791636,...,-69.127889,-1.803900,148.231034,-0.168965,-5.107070,2.357728,-7.464798,0.108440,0.108382,29.743060
2528,2021-04-16 13:35:00-04:00,147.544998,147.619995,147.429993,147.619995,4330.0,1.419841e+06,3271244.0,-0.115133,-955.500004,...,-74.576506,-1.832813,148.220632,0.269994,-9.379228,0.010337,-9.389565,-0.060938,-0.060957,29.663997


In [634]:
hist.columns

Index(['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume', 'volume_adi',
       'volume_obv', 'volume_cmf', 'volume_fi', 'volume_mfi', 'volume_em',
       'volume_sma_em', 'volume_vpt', 'volume_nvi', 'volume_vwap',
       'volatility_atr', 'volatility_bbm', 'volatility_bbh', 'volatility_bbl',
       'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
       'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
       'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'volatility_dcm',
       'volatility_dcw', 'volatility_dcp', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow', 'trend_adx',
       'trend_adx_pos', 'trend_adx_neg', 'trend_vortex_ind_pos',
       'trend_vortex_ind_neg', 'trend_vortex_ind_diff', 'trend_trix',
       'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst',
      

In [635]:
from plotly.subplots import make_subplots


fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Candlestick(
                x=hist.index,
                open=hist['Open'],
                high=hist['High'],
                low=hist['Low'],
                close=hist['Close']),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=hist.index, y=hist["trend_sma_slow"], line=dict(color='orange', width=1)),
    secondary_y=True,
)


fig.show()

In [636]:
def lin_sequel(close, data):
    prices = data['Open'].to_numpy()
    
    prices = np.array_split(prices, 2)
    
    means = [close]
    for price in prices:
        means.append(np.mean(price))
    
    means = np.array(means)
    
    matrix = np.array(
        [
            [0,0,1],
            [1,1,1],
            [4,2,1]
        ]
    )
    
    return np.linalg.solve(matrix, means)[:2]

In [637]:
start = 1


means = lin_sequel(hist.loc[start-1]["Close"], hist[start:start+80])
print(means)

[-0.18926468  0.27769585]


In [638]:
from plotly.subplots import make_subplots

hcut = hist[start:start+80]

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Candlestick(
                x=np.linspace(0,80),
                open=hcut['Open'],
                high=hcut['High'],
                low=hcut['Low'],
                close=hcut['Close']),
    secondary_y=False,
)

def points(x):
    return (x**2 * means[0]) + (x * means[1]) + hist.loc[start-1]["Close"]


fig.add_trace(
    go.Scatter(x=np.linspace(0,80), y=points(np.linspace(0,3,80)), line=dict(color='orange', width=1)),
    secondary_y=False,
)


fig.show()

In [639]:
start_points = np.random.randint(low=170, high=len(hist)-1500, size=3000)

In [640]:
def change(array):
    return array
    #return np.diff(array) / array[:-1]


labels = []
data = []
for point in start_points:
    labels.append(lin_sequel(hist.loc[point-1]["Close"], hist[point:point+80]))
    
    feature = []
    feature.append(change(hist[point-160:point-1]["Close"].to_numpy()))
    feature.append(change(hist[point-160:point-1]["High"].to_numpy()))
    feature.append(change(hist[point-160:point-1]["Low"].to_numpy()))
    feature.append(change(hist[point-160:point-1]["Volume"].to_numpy()))
    
    data.append(feature)
    
    
labels = np.array(labels)
data = np.array(data).reshape(len(data), -1)

In [641]:
from sklearn.preprocessing import StandardScaler
labelScaler = StandardScaler()
labelScaler.fit(labels)
labels_scaled = labelScaler.transform(labels)

dataScaler = StandardScaler()
dataScaler.fit(data)
data_scaled = dataScaler.transform(data)

In [642]:
print(labels_scaled)
print(data_scaled)

[[ 1.69189331 -2.52969396]
 [-1.55764122  0.6881159 ]
 [-1.081073    3.08171761]
 ...
 [ 1.16612614 -0.73435177]
 [ 0.37745171 -0.43940653]
 [-0.19964671  0.34220951]]
[[ 0.95245936  1.04896389  0.96839006 ...  7.23413179  0.79202024
  -0.28741294]
 [ 0.81151919  0.73349541  0.73514149 ... -0.04647748  1.68763721
   0.83047354]
 [-0.61197147 -0.62884199 -0.64667051 ...  0.68281626  0.46455792
  -0.38194549]
 ...
 [-0.43227268 -0.2375889  -0.5123776  ...  0.41750739 -0.43219084
   0.21747682]
 [ 0.72695616  0.73173129  0.73160668 ... -0.57281605 -0.57441813
  -0.39180891]
 [ 1.73467478  1.71515101  1.7829886  ... -0.28213778 -0.44662106
  -0.56788555]]


In [643]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_scaled, labels_scaled, test_size=0.33, random_state=42)

In [662]:
import tensorflow as tf

def get_model():
    inputs = tf.keras.Input(shape=[data_scaled[0].shape[0], 1])
    
    
    x = tf.keras.layers.LSTM(units=50, return_sequences=True)(inputs)
    x = tf.keras.layers.LSTM(units=50)(x)

    
    #x = tf.keras.layers.Dense(30, kernel_regularizer="l2")(inputs)
    #x = tf.keras.layers.Dense(20)(x)
    #x = tf.keras.layers.Dense(10)(x)
    #x = tf.keras.layers.Dropout(0.2)(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(5, kernel_regularizer="l2")(x)
    x = tf.keras.layers.Dense(2, kernel_regularizer="l2")(x)
        
    model = tf.keras.Model(inputs=inputs, outputs=x)
    return model

In [664]:
model = get_model()
model.compile(optimizer="Adam", loss="mae")
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10
63/63 [==============================] - 28s 452ms/step - loss: 0.8615 - val_loss: 0.8313
Epoch 2/10
63/63 [==============================] - 24s 384ms/step - loss: 0.8252 - val_loss: 0.8019
Epoch 3/10
63/63 [==============================] - 25s 397ms/step - loss: 0.7933 - val_loss: 0.7727
Epoch 4/10
63/63 [==============================] - 24s 385ms/step - loss: 0.7607 - val_loss: 0.7086
Epoch 5/10
63/63 [==============================] - 24s 379ms/step - loss: 0.7173 - val_loss: 0.6645
Epoch 6/10
63/63 [==============================] - 28s 446ms/step - loss: 0.7184 - val_loss: 0.6949
Epoch 7/10
63/63 [==============================] - 31s 487ms/step - loss: 0.6958 - val_loss: 0.6598
Epoch 8/10
63/63 [==============================] - 33s 531ms/step - loss: 0.7645 - val_loss: 0.7682
Epoch 9/10
63/63 [==============================] - 29s 457ms/step - loss: 0.7669 - val_loss: 0.7528
Epoch 10/10
63/63 [==============================] - 26s 411ms/step - loss: 0.7583 - val_lo

In [665]:
hist = hist[len(hist)-600:len(hist)]
print(len(hist))
hist = hist.reset_index()

600


In [666]:
from plotly.subplots import make_subplots

start = 422

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Candlestick(
                x=hist.index,
                open=hist['Open'],
                high=hist['High'],
                low=hist['Low'],
                close=hist['Close']),
    secondary_y=False,
)

print(means)

start_points = np.random.randint(low=len(hist)-60, high=len(hist), size=20)

for point in start_points:
    feature = []
    feature.append(change(hist[point-160:point-1]["Close"].to_numpy()))
    feature.append(change(hist[point-160:point-1]["High"].to_numpy()))
    feature.append(change(hist[point-160:point-1]["Low"].to_numpy()))
    feature.append(change(hist[point-160:point-1]["Volume"].to_numpy()))
    
    raw = dataScaler.transform(np.array(feature).reshape(1,-1))
    means = model.predict(raw)
    means = labelScaler.inverse_transform(means)[0]
    
    def points(x):
        return (x**2 * means[0]) + (x * means[1]) + hist.loc[point-1]["Close"]

    
    fig.add_trace(
        go.Scatter(x=np.linspace(point,point+80), y=points(np.linspace(0,3,80)), line=dict(color='orange', width=1)),
        secondary_y=False,
    )



fig.show()

[-0.18926468  0.27769585]
